In [2]:
from IPython.display import display, Math, Latex
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import itertools
import copy
from datetime import datetime

## Group Assignment
### Team Number: 8
### Team Member Names: Xander, Aggam, Charlie
### Team Strategy Chosen: RISKY

# Abstract

Using Pandas, Numpy, and yfinance, to determine the optimal portfolio for weekly returns given a randomly selected dataframe of stock tickers. (CFM 101 Group competition)

The goal of our portfolio generater was to determine the riskiest possible portfolio which we could assemble given various random tickers. To accomplish this, we opted to reduce diversification as much as possilbe. Firstly by reducing the total number of stocks within our portfolio to the minimum allowed by the competition (10), then by placing the majority of our aloted capital (\\$750,000) into as few stocks as possible (20% was the most allowed for any individual stock).

Because we have chosen the risky strategy, we are going to attempt to reduce diversification as much as possible throughout this portfolio. For this reason, you will see the use of MinStocks far more frequently than MaxStocks. Since the fewer stocks we have in our portfolio, the less diversification/saftey we have.

The following are variables given to us for this competition. Additional rules include; permission to buy fractional shares, permission to sell and but immediately, all stocks must be Canadian or US based.

In [3]:
# The following are variables given to us for this competition
MinStocks = 10 #Minimum number of stocks your portfolio must contain
MaxStocks = 22 #Maximum number of stocks your portfolio can contain
MaxWeight = 0.2 #Maximum weight (in decimal percentage) which a single stock can account for 
MinWeight = 0.05 #Minimum weight (in decimal percentage) which a single stock must account for 
Start_date = "2023-01-01" #Starting date of our historical data
End_date = "2023-10-31" #Ending date of our historical data
TradingFee = 4.95 #A fee which we must 
InvestmentAmount = 750000 #Initial capital
MinVolume = 150000 #The minimum monthly volume a stock must have to qualify for a portfolio

Reading in the CSV file containing the names of the tickers. "Tickers.csv" is a file containing a list of strings representing possible tickers for stocks.

In [4]:
tickers = pd.read_csv('Tickers.csv').squeeze()
exchangeCADUSD = yf.download('CADUSD=x', start=Start_date, end=End_date).tz_localize(None)

[*********************100%%**********************]  1 of 1 completed


Creating a function which will convert USD based stocks into CAD (This is so we can more accurately use the correlation function of pandas)

In [5]:
def USDtoCAD(closing):
    #Merges the closing data with the exchange data, then removes all na values (This makes sure each closing price is matched with its dates exchange rate)
    exchanged = pd.concat([closing.to_frame(),exchangeCADUSD["Close"]],axis=1).dropna()

    #Since we are going USD to CAD, we divide by the exchange rate of that day
    converted = exchanged.iloc[:,0]/exchanged.iloc[:,1]
    return converted

validTickers is a function used to determine which tickers can be used in our portfolio out of the given CSV file

In [6]:
## Consumes a series or listof possible stock symboles
def validTickers (tickers):
    ##Creates a new dataframe to hold the valid tickers closing prices
    df = pd.DataFrame()

    #Itterating through the tickers

    for i in tickers:
        #NEW CODE
        if (i not in df.columns) and ((i+".TO") not in df.columns) and (i[:-3] not in df.columns):
            #Creating the ticker objects
            ticker = yf.Ticker(i)

            #Try ensures that even if an error occurs, the code will still run (this gets past the delisted stocks)
            try:
                #Running .fast_info['currency'] on a stock will cause an error and stop any non-listed stocks from being ran
                ticker.fast_info['currency']

                #Download the Daily stock history for the given time periods
                history = ticker.history(start=Start_date, end=End_date).tz_localize(None)

                #Creates a seperate dataframe with the values collected by month
                histByMonth = history['Volume'].to_frame().groupby([(history.index.month)])

                #Drops any months with less than 18 trading days
                histByMonth = histByMonth.filter(lambda x: len(x) > 17)

                #Checks for an average monthly volume greater than 150,000
                if histByMonth.sum().mean() > MinVolume:
                    #Checking if the stock is listed in CAD or USD
                    if ticker.fast_info['currency'] == 'CAD':

                        #Adding the stock to the valid tickers dataframe
                        df[i] = history['Close']

                    elif ticker.fast_info['currency'] == 'USD':

                        #Adding the stocks closing data to the valid tickers dataframe after converting the prices to CAD
                        df[i] = USDtoCAD(history['Close'])
            except:
                #skips and tickers which cause an error
                pass
    return df

"if (i not in df.columns) and ((i+".TO") not in df.columns) and (i[:-3] not in df.columns)"
Will not add a stock if it (or a variant of it listed on another exchange) has already been added to the list of possible tickers. While it would be more precise to scrub the name of the companies directly and compare them, that would be less efficient and has a greater chance of breaking overtime.


Here we are creating a dataframe which is housing the closing prices (in CAD) of all our valid ticker options

In [7]:
stocks_close = validTickers(tickers)

PAPL: Data doesn't exist for startDate = 1672549200, endDate = 1698724800
HDFC.NS: No price data found, symbol may be delisted (1d 2023-01-01 -> 2023-10-31)


Gettings the correlation of all pairs of Tickers

In [9]:
corr = stocks_close.corr()
valid_tickers = list(stocks_close.columns.values)

The next three functions are used to determine which stocks have the highest correlation between them. Because we are going for a risky portfolio, we believe it is best to optimize the correlation between our chosen stocks. (This is an attempt to reduce diversification as much as possible)

In [10]:
#Takes a list of tickers and returns the average correlation among them
def avg_corr(tickers):
    #amount of tickers
    amount = len(tickers)
    #In the case where you only give one ticker, we define
    #The correlation to 1, even though there is nothing to
    #correlated to. This is just here for failsafe as
    #the code would run into a divide by 0 error later for
    #the case where amount = 1.
    if amount == 0:
        return 1;
    sum = 0
    for i in tickers:
        for j in tickers:
            if not i==j:
                sum+=corr[i][j]
    #The average is the sum of each correlation pair divided by the
    #total amount of pairs compared which can be represented as n*(n-1)
    #Yes you can technically reduce the amount of comparisons by 50%
    #but this won't be a problem unless there are like 10000+ stocks.
    average = sum/(amount*(amount-1))
    return average

In [11]:
max_corr = -1
selected = []

#Given an amount to brute force, a list of stocks and a ticker,
#it returns a list of tickers of length n+1, where it includes
#n+1 stocks with the highest average correlation between them,
#given that the ticker given is 1 of the stocks in the list.
#n=amount of tickers to brute force
#selections= list of all tickers you can choose from
#stock= ticker that has to be included in the returned list
def brute_force_nstocks(n, selections, stock):
    #Get all combos of the selections.
    combos = (list(itertools.combinations(valid_tickers,n)))
    max_corr = -1
    selected = []
    #We check each list in combo to see which has the highest
    #average correlation once you add the given ticker
    for i in combos:
        #We don't want to include the given ticker twice
        if not stock in i:
            temp = list(i)
            temp.append(stock)
            corr = avg_corr(temp)
            #If this list's average correlation is larger
            #than the current max, update the max correlation.
            if corr>max_corr:
                max_corr = corr
                selected = temp
    return selected

Estimate_best_corr works to estimate the top 10 most correlated stocks calculated by taking the average correlation between all pairs of stocks. As learned in class, if the prices move in a similar proportion and the same direction, they have a high correlation. High correlation means less diversification, therefore making a risky portfolio (which was the startegy chosen).

In [12]:
#Given how many stocks to brute force for highest average
#correlation and given how many total stocks you want
#in your portfolio, and the list of all options of tickers
#you can choose from, give a rough estimate for
#a portfolio with total amount of stocks with the highest
#average correlation between them.
#We are estimating the best portfolio because
def estimate_best_corr(bf, total, selections):
    #We subtract 1 from the amount we brute force, because
    #we will run the brute_force_nstocks function on each ticker
    #which creates a list of stocks of length n+1, so
    #substituting bf-1 into n will give us
    #a list of length (bf-1)+1 = bf which is ultimately
    #how many stocks we want
    bf = bf-1
    max_corr = -1
    estimated_selected = []
    #We will run the brute force function on each stock.
    for i in selections:
        #brute_force_nstocks on said stock
        selected = brute_force_nstocks(bf, selections, i)
        best_selected = selected
        #For the remaining stocks we need to add to the portfolio
        #we just repeatedly add the stock that creates the highest
        #possible average correlation when added to the list
        #of stocks we got from brute_force_nstocks
        for j in range(total-bf-1):
            temp_max_corr = -1
            #Try adding each stock into the portfolio and see which
            #one gives us the highest average correlation.
            for k in selections:
                #We don't want to add the stock if it
                #is already in our portfolio
                if not k in selected:
                    #Create a temp portfolio with the stock added
                    temp_selected = copy.copy(selected)
                    temp_selected.append(k)
                    corr = avg_corr(temp_selected)
                    #If the average correlation is the highest
                    #we've seen, we update the max correlation
                    #and our best new portfolio
                    if corr>temp_max_corr:
                        temp_max_corr = corr
                        best_selected = copy.copy(temp_selected)
            #Update portfolio
            selected = copy.copy(best_selected)
        corr = avg_corr(selected)
        if corr>max_corr:
            max_corr=corr
            estimated_selected = selected
    return estimated_selected

portfolio = estimate_best_corr(3, 10, valid_tickers)
print(portfolio, avg_corr(portfolio))

['AMZN', 'GOOG', 'GWO.TO', 'ORCL', 'AAPL', 'COST', 'CMCSA', 'CSCO', 'SHOP', 'IBM'] 0.7399828065292531


Here, we are ordering the portfolio based on which stocks will give us the greatest (furthest away from zero) returns

In [13]:
#Returns average weekly return of a stock
def weekly_return(ticker):
    close = stocks_close[ticker]
    returns=close.resample('W').ffill().pct_change()
    return returns.mean()

#Orders the portfolio in terms of which one we want to give more weight,
#and the larger the magnitude of the weekly return is, the higher
#it is prioritized.
def rankings(portfolio):
    #First puts all the tickers and their weekly returns
    #into a dictionary
    rank = {}
    for i in portfolio:
        rank[i] = weekly_return(i)
    #Sort the dictionary
    keys = list(rank.keys())
    values = list(rank.values())
    sorted_index = np.argsort(values)
    ordered = {keys[i]: values[i] for i in sorted_index}

    #Since it is in ascending order at this moment
    #in the case where the returns are negative,
    #we leave the array as it is, as the stock
    #with the lowest return (or highest magnitude)
    #is already ranked first. But if the returns are positive
    #the dict would be fliped, so if the first returns is positive,
    #we flip the dictionary
    first_val = next(iter(ordered.values()))
    if first_val>0:
        ordered = dict(reversed(list(ordered.items())))
    df = pd.DataFrame(ordered.items())
    return df


ranks = rankings(portfolio)
ranks.columns = ["Ticker","Average Weekly Returns(%)"]
ranks

,Ticker,Average Weekly Returns(%)
0,AMZN,0.011268
1,GOOG,0.009832
2,SHOP,0.009183
3,AAPL,0.007355
4,ORCL,0.005558
5,GWO.TO,0.004313
6,COST,0.004069
7,CSCO,0.002951
8,CMCSA,0.002951
9,IBM,0.001542


In [14]:
ranked = ranks["Ticker"].squeeze()
tickerObjects = []
for i in ranked:
    ticker =yf.Ticker(i)
    tickerObjects = tickerObjects+ [ticker]

BestWeights determines how we should distrubute our wealth amoung our chosen stocks. Here, since we are attempting to reduce diversification as much as possible, it is best to place the most capital into as few stocks as possible. For this reason, we have ordered the portfolio into an order of decreasing magnitude of returns. This is going to allow us to put the most capital into the stocks with the greatest expected returns

In [15]:
#Putting the highest possible weight into as few stocks as possible
#returns a list of 10 numbers that represent how much we put into each stock
def BestWeights(numStocks,MaxWeight,MinWeight,InvestmentAmount):
    RemainingAmount = InvestmentAmount - TradingFee*numStocks #subtracting the trading fee each time
    RemainingStocks = numStocks
    MaxAmount = InvestmentAmount*MaxWeight
    MinAmount = InvestmentAmount*MinWeight
    Amounts = []
    #allocates the maximum weight (MaxAmount) to as few stocks as possible
    #until the remaining amount allows it and there are more than one stock remaining
    while RemainingAmount > MinAmount*(RemainingStocks+1) and RemainingStocks > 1:
        RemainingStocks = RemainingStocks - 1
        RemainingAmount = RemainingAmount - MaxAmount
        Amounts = Amounts + [MaxAmount]
    #when the above loop ends (due to insufficient remaining amount or only one stock remaining)
    #we assign the minimum weight (MinAmount) to the remaining stocks
    while RemainingStocks > 1:
        RemainingStocks = RemainingStocks - 1
        RemainingAmount = RemainingAmount - MinAmount
        Amounts = Amounts + [MinAmount]

    Amounts = Amounts + [RemainingAmount]
    sortedAmounts = sorted(Amounts, reverse=True) #sorting the list
    return sortedAmounts


WeightedAmounts = BestWeights(MinStocks,MaxWeight,MinWeight,InvestmentAmount)
WeightedAmounts

[150000.0,
 150000.0,
 150000.0,
 74950.5,
 37500.0,
 37500.0,
 37500.0,
 37500.0,
 37500.0,
 37500.0]

Note that stock #4 has a weight of 9.99% (shown below). This is because we placed the most capital into our highest returning stocks, and the minimum capitital into our lowest returning stocks. The trading fees were taken our of the remaining capital which resulted in stock #4 (our fourth highest expected returning stock) having a negligible difference in weight

This part has nothing to do with how we calculate the portfolio. it is simply putting the portfolio together together.

In [16]:
Portfolio_Final = pd.DataFrame()
Portfolio_Final["Ticker"] = pd.concat([Portfolio_Final,ranks.iloc[:,0]],)

counter = 0
Portfolio_Final["Currency"] = Portfolio_Final["Ticker"]
for i in tickerObjects:
    Portfolio_Final["Currency"][counter] = i.fast_info['currency']
    counter = counter + 1

counter = 0
Portfolio_Final["Shares"] = Portfolio_Final["Ticker"]
exchange = yf.Ticker("CADUSD=x")
for i in tickerObjects:
    if i.fast_info['currency'] == 'USD':
        Portfolio_Final["Shares"][counter] = (WeightedAmounts[counter]*exchange.fast_info["lastPrice"])/i.fast_info['lastPrice']
    else:
        Portfolio_Final["Shares"][counter] = WeightedAmounts[counter]/i.fast_info['lastPrice']
    counter = counter + 1

counter = 0
Portfolio_Final["Value"] = Portfolio_Final["Ticker"]
for i in tickerObjects:
    Portfolio_Final["Value"][counter] = WeightedAmounts[counter]
    counter = counter + 1

counter = 0
Portfolio_Final["Weight"] = Portfolio_Final["Ticker"]
for i in tickerObjects:
    Portfolio_Final["Weight"][counter] = round((WeightedAmounts[counter]/(InvestmentAmount-(10*TradingFee))*100),2)
    counter = counter + 1

#Reindexing
Portfolio_Final.index = [x for x in range(1,MinStocks+1)]

Portfolio_Final

,Ticker,Currency,Shares,Value,Weight
1,AMZN,USD,760.389171,150000.0,20.0
2,GOOG,USD,832.695754,150000.0,20.0
3,SHOP,USD,1533.919314,150000.0,20.0
4,AAPL,USD,292.36147,74950.5,9.99
5,ORCL,USD,239.360523,37500.0,5.0
6,GWO.TO,CAD,862.267155,37500.0,5.0
7,COST,USD,46.77104,37500.0,5.0
8,CSCO,USD,576.055988,37500.0,5.0
9,CMCSA,USD,660.879102,37500.0,5.0
10,IBM,USD,175.246235,37500.0,5.0


In [17]:
data = {"Ticker":Portfolio_Final["Ticker"],"Shares":Portfolio_Final["Shares"]}
Stocks_Finals = pd.DataFrame(data)


Stocks_Finals

,Ticker,Shares
1,AMZN,760.389171
2,GOOG,832.695754
3,SHOP,1533.919314
4,AAPL,292.36147
5,ORCL,239.360523
6,GWO.TO,862.267155
7,COST,46.77104
8,CSCO,576.055988
9,CMCSA,660.879102
10,IBM,175.246235


In [18]:
##Exporting the csv file
myfile = open("Stocks_Group_8.csv", "w")
myfile.write(",Ticker,Shares\n")
for i in range(1, MinStocks+1):
    myfile.write(str(i)+","+Stocks_Finals["Ticker"][i]+","+str(Stocks_Finals["Shares"][i])+ "\n")

myfile.close()

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Alexander Charlie Aggam